# Chapter 10: Data Aggregation and Group Operations

In [3]:
import numpy as np
import pandas as pd

## 10.1 GroupBy Mechanics

**Split-apply-combine** for describing group operations:

1. Split - Data is split into groups based on one or more keys that you provide, performed an a particular axis.
2. Apply - A function is applied to each group, producing a new value.
3. Combine - Resultas of all functions are combined into a result object.

In [4]:
df = pd.DataFrame({'key1': ['a', 'a', 'b', 'b', 'a'],
                    'key2': ['one', 'two', 'one', 'two', 'one'],
                    'data1': np.random.randn(5),
                    'data2': np.random.rand(5)})
df

,key1,key2,data1,data2
0,a,one,0.571884,0.748981
1,a,two,0.999988,0.676610
2,b,one,0.495583,0.639793
3,b,two,-1.589952,0.627048
4,a,one,2.025129,0.342049


Suppose you wanted to compute the mean of the `data1` column using the labels from `key1`.

Option 1: Access `data1` and call `groupby` with the column at `key1`.

In [5]:
grouped = df['data1'].groupby(df['key1'])
grouped

In [6]:
grouped.mean()

key1
a    1.199001
b   -0.547185
Name: data1, dtype: float64

Option 2: Instead pass multiple arrays as a list to get something different.

In [7]:
means = df['data1'].groupby([df['key1'], df['key2']]).mean()
means

key1  key2
a     one     1.298507
      two     0.999988
b     one     0.495583
      two    -1.589952
Name: data1, dtype: float64

In [8]:
means.unstack()

key2,one,two
key1,,
a,1.298507,0.999988
b,0.495583,-1.589952


In thise example, the group keys are all Series.

In [9]:
states = np.array(['Ohio', 'California', 'California', 'Ohio', 'Ohio'])
years = np.array([2005, 2005, 2006, 2005, 2006])

df['data1'].groupby([states, years]).mean()

California  2005    0.999988
            2006    0.495583
Ohio        2005   -0.509034
            2006    2.025129
Name: data1, dtype: float64

Often times the grouping information is found in the same DataFrame so just pass column names as the group keys.

In [10]:
df.groupby('key1').mean()

,data1,data2
key1,,
a,1.199001,0.589213
b,-0.547185,0.633421


> Note: `df['key2']` is not numeric data, a *nuisance column*, which is excluded from the result.

In [11]:
df.groupby(['key1', 'key2']).mean()

data1     data2
key1 key2                    
a    one   1.298507  0.545515
     two   0.999988  0.676610
b    one   0.495583  0.639793
     two  -1.589952  0.627048

A generally useful GroupBy method is `size`, which returns a Series containing group sizes.

In [12]:
df.groupby(['key1', 'key2']).size()

key1  key2
a     one     2
      two     1
b     one     1
      two     1
dtype: int64

### 10.1.1 Iterating Over Groups

The GroupBy object supports iteration, generating a sequence of 2-tuples.

In [13]:
for name, group in df.groupby('key1'):
    print(name)
    print(group)

a
  key1 key2     data1     data2
0    a  one  0.571884  0.748981
1    a  two  0.999988  0.676610
4    a  one  2.025129  0.342049
b
  key1 key2     data1     data2
2    b  one  0.495583  0.639793
3    b  two -1.589952  0.627048


In [14]:
for (k1, k2), group in df.groupby(['key1', 'key2']):
    print((k1, k2))
    print(group)

('a', 'one')
  key1 key2     data1     data2
0    a  one  0.571884  0.748981
4    a  one  2.025129  0.342049
('a', 'two')
  key1 key2     data1    data2
1    a  two  0.999988  0.67661
('b', 'one')
  key1 key2     data1     data2
2    b  one  0.495583  0.639793
('b', 'two')
  key1 key2     data1     data2
3    b  two -1.589952  0.627048


A useful recipe is computing a dict of the data pieces as a one-liner.

In [15]:
pieces = dict(list(df.groupby('key1')))
pieces['b']

,key1,key2,data1,data2
2,b,one,0.495583,0.639793
3,b,two,-1.589952,0.627048


By default, `groupby` groups on `axis=0`, but can group on any axis.

In [16]:
df.dtypes

key1      object
key2      object
data1    float64
data2    float64
dtype: object

In [17]:
grouped = df.groupby(df.dtypes, axis=1)

for dtype, group in grouped:
    print(dtype)
    print(group)

float64
      data1     data2
0  0.571884  0.748981
1  0.999988  0.676610
2  0.495583  0.639793
3 -1.589952  0.627048
4  2.025129  0.342049
object
  key1 key2
0    a  one
1    a  two
2    b  one
3    b  two
4    a  one


### 10.1.2 Selecting a Column or Subset of Columns

Indexing a GroupBy object created from a DataFrame with a column name or array of column names has the effect of column subsetting for aggregation.

    df.groupby('key1')['data1']
    df.groupby('key1')[['data2']]

is the same as:

    df['data1'].groupby(df['key1'])
    df[['data2']].groupby(df['key1'])

As an example, to compute means for just `data2` column.

In [18]:
df.groupby(['key1', 'key2'])[['data2']].mean()

data2
key1 key2          
a    one   0.545515
     two   0.676610
b    one   0.639793
     two   0.627048

In [19]:
s_grouped = df.groupby(['key1', 'key2'])['data2']
s_grouped

In [20]:
s_grouped.mean()

key1  key2
a     one     0.545515
      two     0.676610
b     one     0.639793
      two     0.627048
Name: data2, dtype: float64

### 10.1.3 Grouping with Dicts and Series

In [21]:
people = pd.DataFrame(np.random.randn(5, 5),
                        columns=['a', 'b', 'c', 'd', 'e'],
                        index=['Joe', 'Steve', 'Wes', 'Jim', 'Travis'])
people.iloc[2:3, [1, 2]] = np.nan # Add a few NA values
people

,a,b,c,d,e
Joe,-1.601688,0.163727,-0.567032,0.509771,2.016246
Steve,0.881872,2.054999,1.281609,0.787628,0.402706
Wes,0.062015,NaN,NaN,-0.349731,0.518825
Jim,0.039093,-1.010141,0.524784,1.295755,0.855784
Travis,-1.320966,0.370733,-0.258838,-0.078352,3.742215


Now suppose I want to sum together the columns by group.

In [22]:
mapping = {'a': 'red', 'b': 'red', 'c': 'blue',
            'd': 'blue', 'e': 'red', 'f': 'orange'} # key 'f' is unused and okay

by_column = people.groupby(mapping, axis=1)
by_column.sum()

,blue,red
Joe,-0.057262,0.578284
Steve,2.069237,3.339577
Wes,-0.349731,0.580840
Jim,1.820539,-0.115265
Travis,-0.337189,2.791981


In [23]:
map_series = pd.Series(mapping)
map_series

a       red
b       red
c      blue
d      blue
e       red
f    orange
dtype: object

In [24]:
people.groupby(map_series, axis=1).count()

,blue,red
Joe,2,3
Steve,2,3
Wes,1,2
Jim,2,3
Travis,2,3


### 10.1.4 Grouping with Functions

Suppose you wanted to group by length of the names; pass the `len` function.

In [25]:
people.groupby(len).sum()

,a,b,c,d,e
3,-1.500580,-0.846414,-0.042248,1.455794,3.390854
5,0.881872,2.054999,1.281609,0.787628,0.402706
6,-1.320966,0.370733,-0.258838,-0.078352,3.742215


Mixing functions with arrays, dicts, or Series is okay -> converted to arrays internally.

In [26]:
key_list = ['one', 'one', 'one', 'two', 'two']
people.groupby([len, key_list]).min()

a         b         c         d         e
3 one -1.601688  0.163727 -0.567032 -0.349731  0.518825
  two  0.039093 -1.010141  0.524784  1.295755  0.855784
5 one  0.881872  2.054999  1.281609  0.787628  0.402706
6 two -1.320966  0.370733 -0.258838 -0.078352  3.742215

### 10.1.5 Grouping by Index Levels

For hierarchically indexed datasets, you can aggregate using one of the levels of an axis index.

In [27]:
columns = pd.MultiIndex.from_arrays([['US', 'US', 'US', 'JP', 'JP'],
                                    [1, 3, 5, 1, 3]],
                                    names=['cty', 'tenor'])
hier_df = pd.DataFrame(np.random.randn(4, 5), columns=columns)
hier_df

cty          US                            JP          
tenor         1         3         5         1         3
0      1.134197  0.596682  0.226459 -0.563418 -0.351514
1      1.051869 -1.825875 -1.498879  1.073292  0.484195
2      0.661378 -1.244878  0.342254  0.679564 -0.315856
3      0.177863  0.230781  0.217781  2.119945 -1.254799

To group by level, pass the level number or name using `level`.

In [28]:
hier_df.groupby(level='cty', axis=1).count()

cty,JP,US
0,2,3
1,2,3
2,2,3
3,2,3


## 10.2 Data Aggregation

Aggregation refers to any data transformation that produces scalar values from arrays (`mean`, `count`, `min`, `sum`).

In [29]:
df

,key1,key2,data1,data2
0,a,one,0.571884,0.748981
1,a,two,0.999988,0.676610
2,b,one,0.495583,0.639793
3,b,two,-1.589952,0.627048
4,a,one,2.025129,0.342049


In [30]:
grouped = df.groupby('key1')
grouped['data1'].quantile(0.9)

key1
a    1.820101
b    0.287029
Name: data1, dtype: float64

To use your own aggregation function, pass any function that aggregates an array to `aggregate` or `agg`.

In [31]:
def peak_to_peak(arr):
    return arr.max() - arr.min()

grouped.agg(peak_to_peak)

,data1,data2
key1,,
a,1.453245,0.406932
b,2.085534,0.012745


`describe` also work, even though it is not aggregations, strictly speaking.

In [32]:
grouped.describe()

data1                                                              \
     count      mean       std       min       25%       50%       75%   
key1                                                                     
a      3.0  1.199001  0.746783  0.571884  0.785936  0.999988  1.512559   
b      2.0 -0.547185  1.474696 -1.589952 -1.068568 -0.547185 -0.025801   

               data2                                                    \
           max count      mean       std       min       25%       50%   
key1                                                                     
a     2.025129   3.0  0.589213  0.217088  0.342049  0.509329  0.676610   
b     0.495583   2.0  0.633421  0.009012  0.627048  0.630234  0.633421   

                          
           75%       max  
key1                      
a     0.712795  0.748981  
b     0.636607  0.639793

> Note: Custom aggregation finctions are generally much slower than optimized functions due to extra overhead (function calls, data rearrangement).

### 10.2.1 Column-Wise and Mutiple Function Application

Let's return back to tipping dataset and add a tipping percentage column `tip_pct`.

In [33]:
tips = pd.read_csv('Examples/tips.csv')

# Add tip percentage of total bill
tips['tip_pct'] = tips['tip'] / tips['total_bill']

tips[:6]

,total_bill,tip,smoker,day,time,size,tip_pct
0,16.99,1.01,No,Sun,Dinner,2,0.059447
1,10.34,1.66,No,Sun,Dinner,3,0.160542
2,21.01,3.50,No,Sun,Dinner,3,0.166587
3,23.68,3.31,No,Sun,Dinner,2,0.139780
4,24.59,3.61,No,Sun,Dinner,4,0.146808
5,25.29,4.71,No,Sun,Dinner,4,0.186240


Let's group the `tips` by `day` and `smoker`.

In [34]:
grouped = tips.groupby(['day', 'smoker'])

grouped_pct = grouped['tip_pct']
grouped_pct.agg('mean') # For descriptive statistics
                        # Can pass name of the function as a string

day   smoker
Fri   No        0.151650
      Yes       0.174783
Sat   No        0.158048
      Yes       0.147906
Sun   No        0.160113
      Yes       0.187250
Thur  No        0.160298
      Yes       0.163863
Name: tip_pct, dtype: float64

In [35]:
grouped_pct.agg(['mean', 'std', peak_to_peak])

mean       std  peak_to_peak
day  smoker                                  
Fri  No      0.151650  0.028123      0.067349
     Yes     0.174783  0.051293      0.159925
Sat  No      0.158048  0.039767      0.235193
     Yes     0.147906  0.061375      0.290095
Sun  No      0.160113  0.042347      0.193226
     Yes     0.187250  0.154134      0.644685
Thur No      0.160298  0.038774      0.193350
     Yes     0.163863  0.039389      0.151240

If you pass a list of `(name, function)` tuples, the first element of each tuple will be a DataFrame column name (think of a list of 2-tuples as an ordered mapping).

In [36]:
grouped_pct.agg([('foo', 'mean'), ('bar', np.std)])

foo       bar
day  smoker                    
Fri  No      0.151650  0.028123
     Yes     0.174783  0.051293
Sat  No      0.158048  0.039767
     Yes     0.147906  0.061375
Sun  No      0.160113  0.042347
     Yes     0.187250  0.154134
Thur No      0.160298  0.038774
     Yes     0.163863  0.039389

Suppose we wanted to compute the same three statistics for `tip_pct` and `total_bill`.

In [37]:
functions = ['count', 'mean', 'max']
result = grouped['tip_pct', 'total_bill'].agg(functions)
result

tip_pct                     total_bill                  
              count      mean       max      count       mean    max
day  smoker                                                         
Fri  No           4  0.151650  0.187735          4  18.420000  22.75
     Yes         15  0.174783  0.263480         15  16.813333  40.17
Sat  No          45  0.158048  0.291990         45  19.661778  48.33
     Yes         42  0.147906  0.325733         42  21.276667  50.81
Sun  No          57  0.160113  0.252672         57  20.506667  48.17
     Yes         19  0.187250  0.710345         19  24.120000  45.35
Thur No          45  0.160298  0.266312         45  17.113111  41.19
     Yes         17  0.163863  0.241255         17  19.190588  43.11

In [38]:
result['tip_pct']

count      mean       max
day  smoker                           
Fri  No          4  0.151650  0.187735
     Yes        15  0.174783  0.263480
Sat  No         45  0.158048  0.291990
     Yes        42  0.147906  0.325733
Sun  No         57  0.160113  0.252672
     Yes        19  0.187250  0.710345
Thur No         45  0.160298  0.266312
     Yes        17  0.163863  0.241255

In [39]:
ftuples = [('Durchschnitt', 'mean'), ('Abweichung', np.var)]
grouped['tip_pct', 'total_bill'].agg(ftuples)

tip_pct              total_bill            
            Durchschnitt Abweichung Durchschnitt  Abweichung
day  smoker                                                 
Fri  No         0.151650   0.000791    18.420000   25.596333
     Yes        0.174783   0.002631    16.813333   82.562438
Sat  No         0.158048   0.001581    19.661778   79.908965
     Yes        0.147906   0.003767    21.276667  101.387535
Sun  No         0.160113   0.001793    20.506667   66.099980
     Yes        0.187250   0.023757    24.120000  109.046044
Thur No         0.160298   0.001503    17.113111   59.625081
     Yes        0.163863   0.001551    19.190588   69.808518

Now suppose you wanted to apply different functions to one or more of the columns. Pass a dict to `agg` that contains a mapping of column names to any of the function specifications.

In [40]:
grouped.agg({'tip': np.max, 'size': 'sum'})

tip  size
day  smoker             
Fri  No       3.50     9
     Yes      4.73    31
Sat  No       9.00   115
     Yes     10.00   104
Sun  No       6.00   167
     Yes      6.50    49
Thur No       6.70   112
     Yes      5.00    40

In [41]:
grouped.agg({'tip_pct': ['min', 'max', 'mean', 'std'],
                'size': 'sum'})

tip_pct                               size
                  min       max      mean       std  sum
day  smoker                                             
Fri  No      0.120385  0.187735  0.151650  0.028123    9
     Yes     0.103555  0.263480  0.174783  0.051293   31
Sat  No      0.056797  0.291990  0.158048  0.039767  115
     Yes     0.035638  0.325733  0.147906  0.061375  104
Sun  No      0.059447  0.252672  0.160113  0.042347  167
     Yes     0.065660  0.710345  0.187250  0.154134   49
Thur No      0.072961  0.266312  0.160298  0.038774  112
     Yes     0.090014  0.241255  0.163863  0.039389   40

### 10.2.2 Returning Aggregated Data Without Row Indexes

So far, all aggregated data comes back with an index composed from the unique group key combinations. You can disable this by passing `as_index=False` to `groupby`.

In [42]:
tips.groupby(['day', 'smoker'], as_index=False).mean()

,day,smoker,total_bill,tip,size,tip_pct
0,Fri,No,18.420000,2.812500,2.250000,0.151650
1,Fri,Yes,16.813333,2.714000,2.066667,0.174783
2,Sat,No,19.661778,3.102889,2.555556,0.158048
3,Sat,Yes,21.276667,2.875476,2.476190,0.147906
4,Sun,No,20.506667,3.167895,2.929825,0.160113
5,Sun,Yes,24.120000,3.516842,2.578947,0.187250
6,Thur,No,17.113111,2.673778,2.488889,0.160298
7,Thur,Yes,19.190588,3.030000,2.352941,0.163863


> Note: `reset_index` can also be used but `as_index=False` avoids unncessary computations.

## 10.3 Apply: General split-apply-combine

### 10.3.1 Suppressing the Group Keys

### 10.3.2 Quantile and Bucket Analysis

### 10.3.3 Example: Filling Missing Values with Group-Specific Values

### 10.3.4 Example: Random Sampling and Permutation

### 10.3.5 Example: Group Weighted Average and Correlation

### 10.3.6 Example: Group-Wise Linear Regression

## 10.4 Pivot Tables and Cross-Tabulation

### 10.4.1 Cross-Tabulations: Crosstab

## 10.5